tf.random.normal(
    shape,
    mean=0.0,
    stddev=1.0,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
) -> Outputs random values from a normal distribution

tf.nn.softmax(
    logits,
    axis=None,
    name=None,
    dim=None
)

tf.math.reduce_sum(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the sum of elements across dimensions of a tensor. 
Reduces input_tensor along the dimensions given in axis

tf.math.reduce_mean(
    input_tensor,
    axis=None,
    keepdims=None,
    name=None,
    reduction_indices=None,
    keep_dims=None
) Computes the mean of elements across dimensions of a tensor.
Reduces input_tensor along the dimensions given in axis

In [1]:
import tensorflow as tf
import numpy as np
import json

In [2]:
###
### Citest json files si creeaza lista de cuvinte unice din venues. Scrie rezultatul in fisier.
### return numarul de cuvinte.
###
def creeaza_lista_cuvinte_unice():
    lista_unica_cuvinte = []
    
    ## Citeste lista de venues
    r = open('../DATE/preprocess_venue/lista_venue.txt', 'r')
    lista_venues = json.load(r)
    r.close()
    
    ## Creeaza lista de cuvinte unice din venues
    for venue in lista_venues:
        lista_unica_cuvinte += venue
    lista_unica_cuvinte = set(lista_unica_cuvinte)
    
    w = open('../DATE/lista_unica_cuvinte.txt','w')
    w.write(json.dumps(list(lista_unica_cuvinte)))
    w.close()

    return len(lista_unica_cuvinte)

###
### Citeste lista de cuvinte unice si creeaza dictionar {cuvant:id}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_word2int(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    word2int = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        word2int[cuvant] = i
        
    w = open(out_file,'w')
    w.write(json.dumps(word2int))
    w.close()
    
    
###
### Citeste lista de cuvinte unice si creeaza dictionar {id:cuvant}. Scrie rezultatul in fisier.
### param file_lista_cuvinte: calea spre fisierul de unde se citeste lista de cuvinte unice.
### param out_file: calea spre fisierul unde se va scrie rezultatul
###
def creeaza_int2word(file_lista_cuvinte, out_file):
    # Citeste lista cuvinte
    r = open(file_lista_cuvinte,'r')
    lista_unica_cuvinte = json.load(r)
    r.close()
    
    int2word = {}
    
    for i,cuvant in enumerate(lista_unica_cuvinte):
        int2word[i] = cuvant
        
    w = open(out_file,'w')
    w.write(json.dumps(int2word))
    w.close()

###
### Creeaza un vector de tip one hot.
### param index: pozitia din vector pe care se va pune 1.
### param len_vect: lungimea vectorului.
### return hot_vect: vectorul rezultat.
###
def one_hot(index, len_vect):
    hot_vect = np.zeros(len_vect)
    hot_vect[index] = 1
    return hot_vect

###
###
###
def creeaza_lista_vecini(word2int_file, docs_file, DIM_FEREASTRA, DIM_VOCABULAR):
    # Citeste word2int
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    # Citeste lista documente
    r = open(docs_file, 'r')
    lista_doc = json.load(r)
    r.close()
    
    X_train = [] 
    Y_train = [] 
    
    for doc in lista_doc:
        for word_index, word in enumerate(doc):
            for vecin in doc[max(word_index - DIM_FEREASTRA, 0) : min(word_index + DIM_FEREASTRA, len(doc)) + 1] : 
                if vecin != word:
                    X_train.append(one_hot(word2int[ word ], DIM_VOCABULAR))
                    Y_train.append(one_hot(word2int[ vecin ], DIM_VOCABULAR))

    X_train = np.asarray(X_train)
    Y_train = np.asarray(Y_train)
    
    return X_train, Y_train

def log(word2int_file, vectors, file):
    r = open(word2int_file, 'r')
    word2int = json.load(r)
    r.close()
    
    dict_ = {}
    for elem in word2int:
        dict_[elem] = vectors[word2int[elem]].tolist()
    w = open (file, 'w')
    w.write(json.dumps(dict_))
    w.close()       
    

def train ():
    DIM_VOCABULAR = creeaza_lista_cuvinte_unice()
    #creeaza_word2int('../DATE/lista_unica_cuvinte.txt','../DATE/word2int.txt')
    #creeaza_int2word('../DATE/lista_unica_cuvinte.txt','../DATE/int2word.txt')
    
    DIM_FEREASTRA = 2
    X_train, Y_train = creeaza_lista_vecini('../DATE/word2int.txt', '../DATE/preprocess_venue/lista_venue.txt', DIM_FEREASTRA, DIM_VOCABULAR)
    
    #### Creare retea
    X = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))
    Y_eticheta = tf.placeholder(tf.float32, shape=(None, DIM_VOCABULAR))

    ## Din layer de input in hidden layer
    EMBEDDING_DIM = 50
    W1 = tf.Variable(tf.random_normal([DIM_VOCABULAR, EMBEDDING_DIM]))
    b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
    layer_hidden = tf.add(tf.matmul(X,W1), b1)

    ## Din hidden layer in output layer
    W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, DIM_VOCABULAR]))
    b2 = tf.Variable(tf.random_normal([DIM_VOCABULAR]))
    output_layer = tf.nn.softmax(tf.add( tf.matmul(layer_hidden, W2), b2))
    
    #### Instantiere model tensorflow
    with tf.device('/gpu:0'):
        sess = tf.Session()
        init = tf.global_variables_initializer()
        sess.run(init)

        # Functia de loss -> cross entropy
        cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y_eticheta * tf.log(tf.clip_by_value(output_layer,1e-10,1.0)), reduction_indices=[1]))
        #is actually a horrible way of computing the cross-entropy. In some samples, certain classes could be excluded with certainty after a while, resulting in y_conv=0 for that sample. That's normally not a problem since you're not interested in those, but in the way cross_entropy is written there, it yields 0*log(0) for that particular sample/class. Hence the NaN.
        
        # compute_gradients + apply_gradients = minimize
        learning_rate = 0.01
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

        i = 0
        while(1):
        
            if i > 1500 and i % 100 == 0:
                vectors = sess.run(W1 + b1)
                f_log = '../models/word2vec' + str(i) + '.txt'
                log('../DATE/word2int.txt', vectors, f_log )
                

            sess.run(train_step, feed_dict={X: X_train, Y_eticheta: Y_train})
            l = sess.run(cross_entropy_loss, feed_dict={X: X_train, Y_eticheta: Y_train})
            w = open('../DATE/loss.txt','a')
            w.write(str(l))
            w.write(' ')
            w.close()
            print('loss is : ', l, i)
            i += 1
        
        return vectors

In [ ]:
vectors = train()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
loss is :  22.221954 0
loss is :  22.220766 1
loss is :  22.219576 2
loss is :  22.21839 3
loss is :  22.217196 4
loss is :  22.215982 5
loss is :  22.21476 6
loss is :  22.213533 7
loss is :  22.212307 8
loss is :  22.21108 9
loss is :  22.209858 10
loss is :  22.208628 11
loss is :  22.2074 12
loss is :  22.206173 13
loss is :  22.20495 14
loss is :  22.203724 15
loss is :  22.202494 16
loss is :  22.201239 17
loss is :  22.19999 18
loss is :  22.198734 19
loss is :  22.197481 20
loss is :  22.196232 21
loss is :  22.194983 22
loss is :  22.193735 23
loss is :  22.19249 24
loss is :  22.191244 25
loss is :  22.189999 26
loss is :  22.188751 27
loss is :  22.187508 28
loss is :  22.186256 29
loss is :  22.18501 30
loss is :  22.183765 31
loss is :  22.182526 32
loss is :  22.181282 33
loss is :  22.180044 34
loss is :  22.178804 35
loss is :  22.17756 36
loss is :  2

loss is :  21.58428 312
loss is :  21.580471 313
loss is :  21.576666 314
loss is :  21.572866 315
loss is :  21.569069 316
loss is :  21.565277 317
loss is :  21.561485 318
loss is :  21.557695 319
loss is :  21.553907 320
loss is :  21.55012 321
loss is :  21.54632 322
loss is :  21.542524 323
loss is :  21.538733 324
loss is :  21.534946 325
loss is :  21.531153 326
loss is :  21.527346 327
loss is :  21.523525 328
loss is :  21.51968 329
loss is :  21.515839 330
loss is :  21.511993 331
loss is :  21.508158 332
loss is :  21.504322 333
loss is :  21.500456 334
loss is :  21.49659 335
loss is :  21.4927 336
loss is :  21.4888 337
loss is :  21.484882 338
loss is :  21.480947 339
loss is :  21.477005 340
loss is :  21.473053 341
loss is :  21.469105 342
loss is :  21.465158 343
loss is :  21.461197 344
loss is :  21.457232 345
loss is :  21.453266 346
loss is :  21.449306 347
loss is :  21.445356 348
loss is :  21.441406 349
loss is :  21.43745 350
loss is :  21.4335 351
loss is :  2

loss is :  20.669937 643
loss is :  20.669075 644
loss is :  20.668217 645
loss is :  20.66736 646
loss is :  20.666506 647
loss is :  20.665653 648
loss is :  20.664804 649
loss is :  20.663958 650
loss is :  20.66311 651
loss is :  20.662268 652
loss is :  20.661425 653
loss is :  20.660587 654
loss is :  20.65975 655
loss is :  20.658915 656
loss is :  20.658081 657
loss is :  20.65725 658
loss is :  20.65642 659
loss is :  20.655592 660
loss is :  20.654766 661
loss is :  20.65394 662
loss is :  20.653118 663
loss is :  20.652298 664
loss is :  20.65148 665
loss is :  20.65066 666
loss is :  20.649841 667
loss is :  20.649021 668
loss is :  20.648201 669
loss is :  20.647385 670
loss is :  20.646566 671
loss is :  20.645752 672
loss is :  20.644936 673
loss is :  20.644125 674
loss is :  20.643314 675
loss is :  20.642506 676
loss is :  20.641695 677
loss is :  20.64089 678
loss is :  20.640083 679
loss is :  20.639278 680
loss is :  20.638474 681
loss is :  20.63767 682
loss is : 

loss is :  20.410212 974
loss is :  20.409403 975
loss is :  20.40859 976
loss is :  20.407784 977
loss is :  20.406975 978
loss is :  20.406168 979
loss is :  20.405357 980
loss is :  20.404552 981
loss is :  20.403746 982
loss is :  20.40294 983
loss is :  20.402134 984
loss is :  20.40133 985
loss is :  20.400526 986
loss is :  20.399721 987
loss is :  20.398916 988
loss is :  20.398111 989
loss is :  20.397306 990
loss is :  20.396502 991
loss is :  20.39569 992
loss is :  20.394884 993
loss is :  20.39408 994
loss is :  20.393274 995
loss is :  20.392467 996
loss is :  20.391663 997
loss is :  20.390856 998
loss is :  20.39004 999
loss is :  20.389227 1000
loss is :  20.388409 1001
loss is :  20.387587 1002
loss is :  20.38677 1003
loss is :  20.385948 1004
loss is :  20.38513 1005
loss is :  20.384308 1006
loss is :  20.38349 1007
loss is :  20.382671 1008
loss is :  20.381853 1009
loss is :  20.381035 1010
loss is :  20.380219 1011
loss is :  20.3794 1012
loss is :  20.37858 101

loss is :  20.140621 1293
loss is :  20.139776 1294
loss is :  20.13893 1295
loss is :  20.138083 1296
loss is :  20.137234 1297
loss is :  20.136389 1298
loss is :  20.135542 1299
loss is :  20.134697 1300
loss is :  20.133852 1301
loss is :  20.133005 1302
loss is :  20.132158 1303
loss is :  20.131311 1304
loss is :  20.130468 1305
loss is :  20.12962 1306
loss is :  20.128769 1307
loss is :  20.127916 1308
loss is :  20.127066 1309
loss is :  20.12621 1310
loss is :  20.125349 1311
loss is :  20.124489 1312
loss is :  20.123625 1313
loss is :  20.122765 1314
loss is :  20.121902 1315
loss is :  20.12104 1316
loss is :  20.120178 1317
loss is :  20.11932 1318
loss is :  20.118458 1319
loss is :  20.117598 1320
loss is :  20.11674 1321
loss is :  20.115881 1322
loss is :  20.115019 1323
loss is :  20.114159 1324
loss is :  20.1133 1325
loss is :  20.112442 1326
loss is :  20.111582 1327
loss is :  20.110723 1328
loss is :  20.109869 1329
loss is :  20.10901 1330
loss is :  20.108156 

...

loss is :  10.031356 32407
loss is :  10.031265 32408
loss is :  10.031174 32409
loss is :  10.031084 32410
loss is :  10.030992 32411
loss is :  10.030901 32412
loss is :  10.03081 32413
loss is :  10.03072 32414
loss is :  10.030628 32415
loss is :  10.030538 32416
loss is :  10.030447 32417
loss is :  10.030356 32418
loss is :  10.030266 32419
loss is :  10.030174 32420
loss is :  10.030084 32421
loss is :  10.029991 32422
loss is :  10.029902 32423
loss is :  10.029809 32424
loss is :  10.02972 32425
loss is :  10.029629 32426
loss is :  10.029536 32427
loss is :  10.029447 32428
loss is :  10.029356 32429
loss is :  10.029265 32430
loss is :  10.029174 32431
loss is :  10.029083 32432
loss is :  10.028992 32433
loss is :  10.028901 32434
loss is :  10.0288105 32435
loss is :  10.028719 32436
loss is :  10.028628 32437
loss is :  10.028538 32438
loss is :  10.028447 32439
loss is :  10.028357 32440
loss is :  10.028265 32441
loss is :  10.028174 32442
loss is :  10.028085 32443
los

loss is :  10.003778 32712
loss is :  10.003689 32713
loss is :  10.003598 32714
loss is :  10.003509 32715
loss is :  10.003418 32716
loss is :  10.003328 32717
loss is :  10.003239 32718
loss is :  10.00315 32719
loss is :  10.003059 32720
loss is :  10.00297 32721
loss is :  10.002879 32722
loss is :  10.002789 32723
loss is :  10.0027 32724
loss is :  10.002611 32725
loss is :  10.002521 32726
loss is :  10.002432 32727
loss is :  10.002341 32728
loss is :  10.002252 32729
loss is :  10.002161 32730
loss is :  10.002072 32731
loss is :  10.001983 32732
loss is :  10.001893 32733
loss is :  10.001802 32734
loss is :  10.001714 32735
loss is :  10.001623 32736
loss is :  10.0015335 32737
loss is :  10.001444 32738
loss is :  10.001354 32739
loss is :  10.001264 32740
loss is :  10.001175 32741
loss is :  10.001084 32742
loss is :  10.000995 32743
loss is :  10.000905 32744
loss is :  10.000815 32745
loss is :  10.000726 32746
loss is :  10.000636 32747
loss is :  10.000546 32748
loss

loss is :  9.9757395 33026
loss is :  9.975651 33027
loss is :  9.975563 33028
loss is :  9.975474 33029
loss is :  9.975386 33030
loss is :  9.975296 33031
loss is :  9.975209 33032
loss is :  9.975118 33033
loss is :  9.97503 33034
loss is :  9.974941 33035
loss is :  9.9748535 33036
loss is :  9.974764 33037
loss is :  9.974675 33038
loss is :  9.9745865 33039
loss is :  9.974499 33040
loss is :  9.97441 33041
loss is :  9.974321 33042
loss is :  9.974233 33043
loss is :  9.974144 33044
loss is :  9.974055 33045
loss is :  9.973968 33046
loss is :  9.973879 33047
loss is :  9.973788 33048
loss is :  9.9737 33049
loss is :  9.973611 33050
loss is :  9.973522 33051
loss is :  9.973434 33052
loss is :  9.973346 33053
loss is :  9.973258 33054
loss is :  9.973168 33055
loss is :  9.97308 33056
loss is :  9.972991 33057
loss is :  9.972903 33058
loss is :  9.972815 33059
loss is :  9.972726 33060
loss is :  9.972637 33061
loss is :  9.9725485 33062
loss is :  9.972461 33063
loss is :  9.

loss is :  9.947811 33343
loss is :  9.947724 33344
loss is :  9.947637 33345
loss is :  9.947549 33346
loss is :  9.947461 33347
loss is :  9.947374 33348
loss is :  9.947287 33349
loss is :  9.947199 33350
loss is :  9.947111 33351
loss is :  9.947024 33352
loss is :  9.946937 33353
loss is :  9.946849 33354
loss is :  9.946761 33355
loss is :  9.946673 33356
loss is :  9.946587 33357
loss is :  9.9465 33358
loss is :  9.946411 33359
loss is :  9.946324 33360
loss is :  9.946237 33361
loss is :  9.946149 33362
loss is :  9.946062 33363
loss is :  9.945975 33364
loss is :  9.945887 33365
loss is :  9.9458 33366
loss is :  9.945712 33367
loss is :  9.945624 33368
loss is :  9.945538 33369
loss is :  9.945449 33370
loss is :  9.945362 33371
loss is :  9.945276 33372
loss is :  9.945189 33373
loss is :  9.945102 33374
loss is :  9.945014 33375
loss is :  9.944927 33376
loss is :  9.944839 33377
loss is :  9.944752 33378
loss is :  9.944665 33379
loss is :  9.944577 33380
loss is :  9.944

loss is :  9.920326 33659
loss is :  9.920239 33660
loss is :  9.920153 33661
loss is :  9.920067 33662
loss is :  9.919979 33663
loss is :  9.919892 33664
loss is :  9.919806 33665
loss is :  9.91972 33666
loss is :  9.919632 33667
loss is :  9.919546 33668
loss is :  9.91946 33669
loss is :  9.919374 33670
loss is :  9.919286 33671
loss is :  9.9192 33672
loss is :  9.919114 33673
loss is :  9.919027 33674
loss is :  9.918941 33675
loss is :  9.918854 33676
loss is :  9.918767 33677
loss is :  9.91868 33678
loss is :  9.918594 33679
loss is :  9.918508 33680
loss is :  9.918422 33681
loss is :  9.918335 33682
loss is :  9.918249 33683
loss is :  9.918161 33684
loss is :  9.918076 33685
loss is :  9.917989 33686
loss is :  9.917902 33687
loss is :  9.917816 33688
loss is :  9.917729 33689
loss is :  9.917643 33690
loss is :  9.917556 33691
loss is :  9.91747 33692
loss is :  9.917383 33693
loss is :  9.917296 33694
loss is :  9.917211 33695
loss is :  9.917124 33696
loss is :  9.91703

In [ ]:
loss is :  5.676243 18087